In [179]:
%matplotlib inline
# imported necessary libraries
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import cross_val_score
#from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import model_selection
#from sklearn import cross_validation
from sklearn.model_selection import cross_validate


# using the SQLite Table to read data.
con = sqlite3.connect('database.sqlite') 
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 limit 1000
""", con) 

def partition(x):
    if x < 3:
        return 1
    return 0




In [ ]:
https://www.kaggle.com/premvardhan/amazon-fine-food-reviews-analysis-naive-bayes

In [180]:


#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative


In [181]:
filtered_data.shape #looking at the number of attributes and size of the data
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,0,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,1,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,0,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [184]:


#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')



In [237]:
final = sorted_data.drop_duplicates(subset = {"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(998, 10)

In [238]:
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

99.8

In [239]:


final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]



In [240]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(998, 10)


0    841
1    157
Name: Score, dtype: int64

In [241]:
import re
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;    

0
I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!


In [242]:
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'ours', 'yourself', 'from', 'own', 'and', 'don', 'yourselves', 'under', 'should', 'than', 't', 'what', 'did', "should've", "needn't", "shouldn't", 'for', 'some', 'so', 'there', 'haven', 'doing', 'd', 'she', 'herself', 'he', 'ma', 'ain', "won't", 'how', 'does', 'ourselves', 've', 'above', 'isn', 'do', 'am', 'off', 'then', 'any', "that'll", 'has', "wouldn't", 'him', 'but', "you'd", 'the', 'very', 'you', 'all', 'not', 'couldn', 'in', "mustn't", "she's", 'over', "it's", 'between', 'hers', "hasn't", 'be', 'o', 're', 'been', 'had', 'why', 'shan', 'before', 'being', "don't", 'only', 'hasn', 'your', 'on', 'mightn', 'who', 'here', 'into', 'further', 'same', 'at', 'to', 'itself', 'are', 'theirs', 'needn', 'mustn', 'weren', 'doesn', 'or', 'was', 'were', 'can', 'each', 'yours', 'm', 's', 'about', "you'll", 'aren', 'didn', 'y', 'because', 'shouldn', 'now', 'll', "doesn't", 'a', 'below', 'other', "wasn't", 'her', "didn't", 'most', 'that', 'i', 'its', 'more', "aren't", 'against', 'we', 'when', 'me',

In [243]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [244]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 

In [245]:
final.head(3) #below the processed review can be seen in the CleanedText Column 


# store final table into an SQlLite table for future.
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

In [246]:
con = sqlite3.connect("final.sqlite")
cleaned_data = pd.read_sql_query("select * from Reviews", con)

In [247]:
cleaned_data.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,10,11,B0001PB9FE,A3HDKO7OW0QNK4,Canadian Fan,1,1,0,1107820800,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...,b'dont know cactus tequila uniqu combin ingred...
1,232,253,B0002567IW,A1SSKFPX72MSMR,"Janna M. Sicard ""missjanna""",0,0,1,1344556800,Sad outcome,"Five minutes in, one tentacle was bitten off, ...",b'five minut one tentacl bitten ball insid cra...
2,171,188,B00029XIZI,A2S72TUJDQUBMH,Nadia,0,0,0,1236124800,Miracle,My Scotties were full of hot spots and when I ...,b'scotti full hot spot use within week hot spo...
3,932,1011,B0002MKFEM,A3QLX72AO0DD5Z,Carlito Picache,1,2,0,1226361600,Way too salty,I tried this and I found it too salty.<br />Pl...,b'tri found salti plus ate fish sauc fish real...
4,893,969,B0002XIB2Y,A3NV17B17PFB7G,Susan,3,3,0,1228176000,WONDERFUL gravy!,"This gravy mix is excellent ... except, don't ...",b'gravi mix excel except dont use water call u...


In [248]:
cleaned_data["Score"].value_counts()



0    841
1    157
Name: Score, dtype: int64

In [255]:
# To randomly sample 100k points from both class

data_pos = cleaned_data[cleaned_data["Score"] == 0].sample(n = 100)
data_neg = cleaned_data[cleaned_data["Score"] == 1].sample(n = 100)
final_100k = pd.concat([data_pos, data_neg])
final_100k.shape

(200, 12)

In [250]:
final_100k["Time"] = pd.to_datetime(final_100k["Time"], unit = "s")
final_100k = final_100k.sort_values(by = "Time")
final_100k.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
86,672,724,B000G6MBX2,A1ABDQ02BCIGYQ,HM,3,7,1,2007-03-14,"Not, my favorite chip","I did not like these potatoe chips. It's hot, ...",b'like potato chip hot sweet overwhelm tast ga...
117,664,715,B000G6MBX2,A14B8M117EUBLK,Shelly Kenyon,0,0,0,2007-06-01,Best tortilla chips ever!!!,We like these chips for salsa and quacomole be...,b'like chip salsa quacomol better cip market e...
240,460,498,B000G6RYNE,AJQD2WWJYOYFQ,bubbles,0,0,0,2007-08-09,"Tangy, spicy, and sweet- oh my!",Kettle Chips Spicy Thai potato chips have the ...,b'kettl chip spici thai potato chip perfect am...
415,734,791,B000UZMJZO,A2V71J7J3DTB5B,Vehred,2,3,1,2007-11-27,If I could give a rating under one-star...,This tea carries the flavor one would expect f...,b'tea carri flavor one would expect terribl in...
515,366,399,B001ELL6O8,ALSAOZ1V546VT,"A Research It Maven ""Just the facts please""",1,1,0,2008-01-13,Arrowhead Mills whole grain buttermilk Pancake...,HEY! These are GREAT Waffles and Pancakes! We ...,b'hey great waffl pancak add mix tho recipi ba...


In [217]:
# 100k data which will use to train model after vectorization
X = final_100k["CleanedText"]
print("shape of X:", X.shape)

shape of X: (100,)


In [218]:
# class label
y = final_100k["Score"]
print("shape of y:", y.shape)

shape of y: (100,)


In [269]:
# Run this program on your local python 
# interpreter, provided you have installed 
# the required libraries. 

# Importing the required packages 
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
#from sklearn.cross_validation import train_test_split 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
#from sklearn import cross_validation

# Function importing Dataset 
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle=False)
y_train

532    0
304    0
517    1
239    0
341    0
84     0
528    1
349    0
105    0
192    1
198    1
322    0
166    1
163    1
275    0
81     0
674    0
161    1
456    1
511    0
281    0
221    0
417    1
627    1
164    1
366    1
570    1
45     1
481    1
549    1
      ..
504    1
538    0
595    0
302    1
606    0
67     1
465    0
510    0
402    1
707    0
883    1
492    1
739    1
889    0
283    0
183    1
859    0
667    1
892    0
245    0
787    0
142    0
736    1
564    1
168    1
691    1
620    0
963    1
472    0
181    1
Name: Score, dtype: int64

In [272]:
from sklearn.feature_extraction.text import CountVectorizer 

bow = CountVectorizer()
#X_train = bow.fit_transform(X_train)
y_train = bow.fit_transform(y_train)

AttributeError: 'numpy.int64' object has no attribute 'lower'

In [236]:
y_train

322    0
510    0
966    0
481    1
971    1
84     0
620    0
691    1
365    1
674    0
707    0
165    1
349    0
592    1
45     1
511    0
921    1
366    1
736    1
163    1
164    1
239    0
161    1
66     1
105    0
932    1
528    1
168    1
457    1
873    1
      ..
981    0
538    0
892    0
402    1
521    0
667    1
874    1
594    1
984    1
245    0
564    1
980    0
458    1
549    1
304    0
739    1
221    0
517    1
627    1
854    1
928    0
911    0
842    1
881    1
281    0
787    0
901    0
275    0
986    1
492    1
Name: Score, dtype: int64

In [229]:
def train_using_gini(X_train, X_test, y_train): 

	# Creating the classifier object 
	clf_gini = DecisionTreeClassifier(criterion = "gini", 
			random_state = 100,max_depth=3, min_samples_leaf=5) 

	# Performing training 
	clf_gini.fit(X_train, y_train) 
	return clf_gini 
	
# Function to perform training with entropy. 
def tarin_using_entropy(X_train, X_test, y_train): 

	# Decision tree with entropy 
	clf_entropy = DecisionTreeClassifier( 
			criterion = "entropy", random_state = 100, 
			max_depth = 3, min_samples_leaf = 5) 

	# Performing training 
	clf_entropy.fit(X_train, y_train) 
	return clf_entropy 


# Function to make predictions 
def prediction(X_test, clf_object): 

	# Predicton on test with giniIndex 
	y_pred = clf_object.predict(X_test) 
	print("Predicted values:") 
	print(y_pred) 
	return y_pred 
	
# Function to calculate accuracy 
def cal_accuracy(y_test, y_pred): 
	
	print("Confusion Matrix: ", 
		confusion_matrix(y_test, y_pred)) 
	
	print ("Accuracy : ", 
	accuracy_score(y_test,y_pred)*100) 
	
	print("Report : ", 
	classification_report(y_test, y_pred)) 

# Driver code 
def main(): 
	
	# Building Phase 
	clf_gini = train_using_gini(X_train, X_test, y_train) 
	clf_entropy = tarin_using_entropy(X_train, X_test, y_train) 
	
	# Operational Phase 
	print("Results Using Gini Index:") 
	
	# Prediction using gini 
	y_pred_gini = prediction(X_test, clf_gini) 
	cal_accuracy(y_test, y_pred_gini) 
	
	print("Results Using Entropy:") 
	# Prediction using entropy 
	y_pred_entropy = prediction(X_test, clf_entropy) 
	cal_accuracy(y_test, y_pred_entropy) 
	
	
# Calling main function 
if __name__=="__main__": 
	main() 


ValueError: could not convert string to float: b'secret high fructos corn syrup detriment health unfortun also secret replac form sugar disturb larg number manufactur food even wors high fructos corn syrup made genet modifi corn sinc late hfcs replac regular tabl sugar honey similar sweeten practic everyth prolong consumpt hfcs topic much debat learn caus damag bodi high fructos corn syrup danger signific risk weight gain obes increas risk develop diabet hypertens elev bad cholesterol level liver damag mercuri exposur hfcs kellogg manufactur know slowli get rid sweetner look cover post rain bran advertis contain high fructos corn syrup applaud high fructos corn syrup made genet modifi corn monsanto use system pesticid seed caus coloni collaps disord bee bee keeper avoid product hfcs save famili bee watch vanish bee food matter food inc sold amazon excel movi chang health dramat way think eat'